In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report, f1_score, recall_score, confusion_matrix
import sklearn
import math
import warnings
import seaborn as sns
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA


warnings.filterwarnings("ignore")


In [2]:
data = pd.read_csv("wine_quality.csv")

print(data.head())

columns  = data.columns


   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
4            7.4              0.70         0.00             1.9      0.076   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 25.0                  67.0   0.9968  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65   
3                 17.0                  60.0   0.9980  3.16       0.58   
4                 11.0                  34.0   0.9978  3.51       0.56   

   alcohol  quality  
0      9.4        5  
1      9.8        5  
2      9.8        5 

In [3]:
# Missing values

def change_mean():
  for i in data.columns[data.isnull().any(axis=0)]: 
    data[i] = data.groupby("quality")[i].transform(lambda x: x.fillna(x.mean()))

def drop_cols(data):
    for i in range(data.shape[0]):
        if (math.isnan(data.loc[i,'ph']) or math.isnan(data.loc[i, 'Sulfate']) or math.isnan(data.loc[i, "Trihalomethanes"])):
            data = data.drop(i, axis = 0)
    return data

# Outliers

def change_mean_out():
  for col in columns:
      mean = np.mean(data.loc[:, col])
      std = np.std(data.loc[:, col])
      for i in range(data.shape[0]):
        if data.loc[i,col] > mean + 2 * std:
          data[col] = data.groupby("quality")[col].transform(lambda x: x.fillna(x.mean()))

def drop_out():
    for col in columns:
      mean = np.mean(data.loc[:, col])
      std = np.std(data.loc[:, col])
      for i in range(data.shape[0]):
        if data.loc[i,col] > mean + 2 * std:
          data.drop(i, axis = 0)

#data = drop_cols(data)
change_mean()
change_mean_out()

print(data.isnull().sum())
print(data.shape)

pca = PCA(n_components=3)
pca.fit(data)

y = data["quality"]
x = data.drop(["quality"], axis = 1)


fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64
(1599, 12)


In [4]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3)

scaler = StandardScaler()
scaler.fit(x_train)

x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

smote = SMOTE()
x_train, y_train = smote.fit_resample(x_train, y_train)

In [5]:
mlp = MLPClassifier()

search_space = { "max_iter": [50, 100, 200],
                "hidden_layer_sizes" : [(50,), (100,)]}

model1 = GridSearchCV(mlp, search_space)

model1.fit(x_train, y_train) 

y_pred = model1.predict(x_test)

In [6]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           3       0.00      0.00      0.00         1
           4       0.15      0.24      0.18        17
           5       0.66      0.63      0.65       205
           6       0.57      0.43      0.49       202
           7       0.27      0.49      0.35        47
           8       0.20      0.38      0.26         8

    accuracy                           0.51       480
   macro avg       0.31      0.36      0.32       480
weighted avg       0.56      0.51      0.53       480



In [ ]:
scores = cross_validate(model1, x_train, y_train, cv=5, scoring=['accuracy', 'f1_weighted', 'recall_weighted'])

print(np.mean(scores['test_accuracy']), np.mean(scores['test_f1_weighted']), np.mean(scores['test_recall_weighted']))

In [ ]:
y_pred = model1.predict(x_test)

print(accuracy_score(y_test, y_pred), f1_score(y_test, y_pred, average="weighted"), recall_score(y_test, y_pred, average="weighted"))
print()

confusion_matrix(y_test, y_pred)